# 🚀 VerifDoc Engine - Colab Environment

Exécutez les cellules les unes après les autres (Bouton ▶️) pour lancer l'analyse.

In [ ]:
# 1. Installation des Dépendances (OCR & PDF Tools)
!apt-get install -y poppler-utils tesseract-ocr libtesseract-dev libgl1-mesa-glx
!pip install fastapi uvicorn python-multipart pikepdf Pillow pytesseract numpy opencv-python-headless fpdf requests google-generativeai

In [ ]:
# 2. Importation des Fichiers (Code Interactif)
from google.colab import files
import os

print("👇 Cliquez sur le bouton 'Choisir les fichiers' ci-dessous.")
print("⚠️ Veuillez sélectionner DEUX fichiers en même temps (Ctrl+Clic) :")
print("   1. verifdoc_colab_pack_v2.zip (Le moteur)")
print("   2. Votre fichier de données (data.mdb, data.mdb.zip OU dataset.zip)")

uploaded = files.upload()

for fn in uploaded.keys():
  print(f'✅ Reçu : {fn} ({len(uploaded[fn])} octets)')

In [ ]:
# 3. Décompression et Préparation
import os

# Unzip Engine
if os.path.exists('verifdoc_colab_pack_v2.zip'):
    !unzip -o verifdoc_colab_pack_v2.zip -d verifdoc_engine
elif os.path.exists('verifdoc_colab_pack.zip'):
    !unzip -o verifdoc_colab_pack.zip -d verifdoc_engine
else:
    print("❌ ERREUR : verifdoc_colab_pack.zip manquant ! Relancez l'étape 2.")

# Handle Data (Support de data.mdb.zip)
if os.path.exists('data.mdb.zip'):
    print("📦 Archive data.mdb.zip détectée. Décompression...")
    !unzip -o data.mdb.zip
    print("✅ Décompression terminée (data.mdb extrait).")


if os.path.exists('data.mdb'):
    print("📦 Données MDB détectées. Extraction en cours...")
    !mkdir -p verifdoc_engine/DATASET_REAL
    !mv data.mdb verifdoc_engine/DATASET_REAL/
    !python verifdoc_engine/scripts/carve_mdb.py
    !mkdir -p verifdoc_engine/DATASET
    # Copy extracted files
    !cp verifdoc_engine/DATASET_REAL/extracted_carved/* verifdoc_engine/DATASET/
    print("✅ Extraction terminée.")

elif os.path.exists('dataset.zip'):
    print("📦 Données ZIP détectées. Décompression...")
    !unzip -o dataset.zip -d verifdoc_engine/DATASET
    print("✅ Décompression terminée.")

else:
    print("⚠️ ATTENTION : Aucun fichier de données trouvé (data.mdb ou dataset.zip). L'analyse risque d'être vide.")

In [ ]:
# 4. Démarrage du Serveur API (Arrière-plan)
import subprocess
import time
import os

os.chdir('verifdoc_engine')
print("🚀 Démarrage du moteur VerifDoc...")
proc = subprocess.Popen(['uvicorn', 'main:app', '--host', '0.0.0.0', '--port', '8000'])
time.sleep(5)
print("✅ Serveur en ligne.")

In [ ]:
# 5. Lancement de l'Analyse Parallèle
!python scripts/batch_runner_parallel.py

# 🔬 Module de Recherche : Benchmark "Dual-Analysis"
Ce module exécute le script de test spécifique demandé pour valider l'approche Hybride (Pixels + Logique).

In [ ]:
import google.generativeai as genai
from PIL import Image
import random
import os
import glob

def calculer_decision_finale(image_path, label_reel, secteur="BANQUE"):
    # --- MOT D'APPRENTISSAGE : INFERENCE ---
    # 1. Simulation du score de ton modèle local (Pixels)
    # Dans un vrai projet, ici on appelle ton modèle entraîné
    score_pixels = random.uniform(0.1, 0.9) if label_reel == 1 else random.uniform(0.0, 0.3)
    
    # 2. Appel à Gemini pour le score logique (Sémantique)
    try:
        img = Image.open(image_path)
    except: return None

    prompt = "Analyse la cohérence de ce document. Réponds uniquement par un score de suspicion entre 0 (sain) et 100 (fraude)."
    
    # On simule ici la réponse pour l'exemple, mais tu peux activer le vrai modèle
    # response = model.generate_content([prompt, img])
    score_logique = 75 if label_reel == 1 else 10 

    # --- MOT D'APPRENTISSAGE : PONDÉRATION ---
    # 3. Calcul du Score de Risque (S_risk) sur 100
    # Formule Utilisateur: (score_pixels * 40) + (score_logique * 0.6)
    s_risk = (score_pixels * 40) + (score_logique * 0.6)
    
    # --- MOT D'APPRENTISSAGE : SEUIL (THRESHOLD) ---
    # 4. Décision d'Acceptabilité
    seuil = 15 if secteur == "BANQUE" else 30
    verdict = "REJETÉ" if s_risk > seuil else "ACCEPTÉ"
    
    return {
        "Fichier": os.path.basename(image_path),
        "Risque": round(s_risk, 2),
        "Verdict": verdict,
        "Seuil_Appliqué": seuil,
        "Secteur": secteur
    }

# Exécution sur 5 images du Dataset réel extrait
print("--- Lancement du Benchmark Dual-Analysis ---")
files = glob.glob("verifdoc_engine/DATASET/*.jpg")[:5]
for f in files:
    res = calculer_decision_finale(f, label_reel=1) # 1 = Simulons une anomalie pour le test
    if res:
        print(res)

In [ ]:
# 6. Téléchargement des Rapports
!zip -r reports.zip static/reports
from google.colab import files
files.download('reports.zip')